In [1]:
from src.relationship_agent import graph




In [2]:
graph.invoke({"question": "How should I love my girlfired more?"})

{'answer': 'Listen, appreciate, support her dreams, and create cherished memories together.'}

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

async def mcp_tools_node():
    client = MultiServerMCPClient({
        "agent": {
            "transport": "streamable_http",
            "url": "http://localhost:2024/mcp", 
        }
    })
    tools = await client.get_tools() 
    return tools
tools = await mcp_tools_node()
print(tools)

[StructuredTool(name='relationship_agent', args_schema={'properties': {'question': {'title': 'Question', 'type': 'string'}}, 'required': ['question'], 'title': 'InputState', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x10e95bba0>)]


In [4]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage
tool_node = ToolNode(tools)
AI_message = AIMessage(
            content="",
            tool_calls=[
                {
                    "name": "relationship_agent",
                    "args": {"question": "can you tell me how to make my partner happy?"},
                    "id": "tool_call_id_1",
                    "type": "tool_call_1",
                }
            ]
            
        )
result = await tool_node.ainvoke({"messages": [AI_message]})
print(result["messages"][-1].content)

{'answer': 'Communicate openly, listen deeply, show affection, support dreams, appreciate them always.'}


In [ ]:
from langsmith import Client

client = Client()
dataset_name = "Chatbot QA Dataset"

try:
    dataset = client.create_dataset(
        dataset_name=dataset_name, description="A test dataset for relationship agent.", 
    )

    client.create_examples(
            dataset_id=dataset.id,
            examples = [
            {
                "inputs": {"question": "How do I rebuild trust after a breakup?"},
                "outputs": {"answer": "Rebuilding trust takes time, honest communication, and consistent actions."},
            },
            {
                "inputs": {"question": "What should I do if my partner and I keep arguing?"},
                "outputs": {"answer": "Try to identify the root cause and communicate calmly and openly."},
            },
            {
                "inputs": {"question": "Is it normal to have doubts in a long-term relationship?"},
                "outputs": {"answer": "Yes, occasional doubts are common and worth discussing respectfully."},
            },
            {
                "inputs": {"question": "How can I improve communication with my partner?"},
                "outputs": {"answer": "Practice active listening, be honest, and avoid blame during conversations."},
            },
            {
                "inputs": {"question": "What are signs of a healthy relationship?"},
                "outputs": {"answer": "Trust, respect, open communication, and emotional support are key signs."},
            }
        ]
        )
except Exception as e:
    dataset = client.read_dataset(dataset_name=dataset_name)
    print("dataset already exists. loaded it...")

dataset already exists. loaded it...


{'example_ids': ['ed52d3bd-25fd-4f89-801c-feeefeb0a6b8',
  '601b933f-f518-4b0c-8c58-80a85b69fea2',
  '143c92e4-0a61-4727-95c6-a568ab4dca63',
  '2cef39bb-06a2-4861-86ad-abbe4657d891',
  '27959b26-1d0c-4eef-9f38-937fb1595802'],
 'count': 5}

In [6]:
from langsmith import wrappers
import openai
import os

Gemini = openai.OpenAI( api_key=os.getenv("GEMINI_API_KEY"),base_url="https://generativelanguage.googleapis.com/v1beta/")

llm_judge = wrappers.wrap_openai(Gemini)

eval_instructions = "You are an expert evaluating answers to questions."

async def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    user_content = f"""Question: {inputs['question']}
    Expected Answer: {reference_outputs['answer']}
    Predicted Answer: {outputs['response']}
    Is the predicted answer correct? Respond with CORRECT or INCORRECT."""
    
    response = llm_judge.chat.completions.create(
        model="gemini-2.5-flash",
        temperature=0,
        messages=[
            {"role": "system", "content": eval_instructions},
            {"role": "user", "content": user_content},
        ],
    ).choices[0].message.content

    return response.strip() == "CORRECT"

async def concision(outputs: dict, reference_outputs: dict) -> bool:
    return len(outputs["response"]) < 2 * len(reference_outputs["answer"])

In [7]:
from langchain_core.messages import AIMessage
import ast

response_str = "{'answer': 'Listen attentively, communicate openly, and show appreciation consistently. Support their passions, spend quality time, and be truly present for them.'}"
response_dict = ast.literal_eval(response_str)

async def ls_target(inputs: dict) -> dict:
    AI_message = AIMessage(
                            content="",
                            tool_calls=[
                                {
                                    "name": "relationship_agent",
                                    "args": {"question": f"{inputs["question"]}"},
                                    "id": "tool_call_id_1",
                                    "type": "tool_call_1",
                                }
                            ]
                            
                        )
   
    result = await tool_node.ainvoke({"messages": [AI_message]})
    return {"response": ast.literal_eval(result["messages"][-1].content)["answer"]}

In [8]:
experiment_results = await client.aevaluate(
    ls_target,
    data=dataset_name,
    evaluators=[concision, correctness],
    experiment_prefix="gemini-chatbot-eval"
)

/Users/pankajchandravanshi/mcp/langgraph-evals/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gemini-chatbot-eval-29ab890e' at:
https://smith.langchain.com/o/cb35aec6-2b3b-4044-bfca-43dbfad00211/datasets/48a59600-0eb4-46ee-82a9-ccbbbd1a8a3b/compare?selectedSessions=8dbd4923-feaf-4fab-9749-d92e864e2cd4




6it [00:33,  5.66s/it]Error running target function: invalid syntax (<unknown>, line 1)
Traceback (most recent call last):
  File "/Users/pankajchandravanshi/mcp/langgraph-evals/.venv/lib/python3.13/site-packages/langsmith/evaluation/_arunner.py", line 1262, in _aforward
    await fn(*args, langsmith_extra=langsmith_extra)
  File "/var/folders/c9/h7x3gp7x57n7kz8n91yjhw6w0000gn/T/ipykernel_67812/4290996547.py", line 22, in ls_target
    return {"response": ast.literal_eval(result["messages"][-1].content)["answer"]}
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/ast.py", line 68, in literal_eval
    node_or_string = parse(node_or_string.lstrip(" \t"), mode='eval')
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/ast.py", line 54, in parse
    return compile(source, filename, mode, flags,
                   _feature_version=feature_version, optimize=optimize)
  File

In [11]:
experiment_results.to_pandas()

,inputs.question,outputs.response,error,reference.answer,feedback.concision,feedback.correctness,execution_time,example_id,id,feedback.awrapper,outputs.output
0,Is it normal to have doubts in a long-term rel...,"Yes, completely normal. Relationships evolve; ...",None,"Yes, occasional doubts are common and worth di...",True,True,3.199132,143c92e4-0a61-4727-95c6-a568ab4dca63,112ec2c9-bd36-48df-b39c-bd45c1f1f0f7,NaN,NaN
1,What are signs of a healthy relationship?,"Trust, respect, open communication, mutual sup...",None,"Trust, respect, open communication, and emotio...",True,True,4.035334,27959b26-1d0c-4eef-9f38-937fb1595802,8526a294-00d1-4724-b729-47d334483ba5,NaN,NaN
2,How can I improve communication with my partner?,"Listen actively, express needs clearly, valida...",None,"Practice active listening, be honest, and avoi...",True,True,2.409553,2cef39bb-06a2-4861-86ad-abbe4657d891,04889565-79e6-4e33-bea2-54ef5afd75b9,NaN,NaN
3,What should I do if my partner and I keep argu...,"Communicate, listen, understand, compromise. I...",None,Try to identify the root cause and communicate...,True,True,3.586426,601b933f-f518-4b0c-8c58-80a85b69fea2,8230e1dc-cef8-4c6a-b219-2fea08544862,NaN,NaN
4,How do I rebuild trust after a breakup?,"Consistent actions, honest communication, and ...",None,"Rebuilding trust takes time, honest communicat...",True,True,2.685530,ed52d3bd-25fd-4f89-801c-feeefeb0a6b8,d083a9ba-83e7-4eda-a942-ab0d1ed233ae,NaN,NaN
5,What should I do if my partner and I keep argu...,"Understand triggers, communicate respectfully,...",None,Try to identify the root cause and communicate...,True,NaN,3.673528,54790224-7632-4676-becf-1d1405cf513a,5cb5c173-1b9b-4ed5-80aa-5ec0fc59768f,NaN,NaN
6,How do I rebuild trust after a breakup?,NaN,"SyntaxError('invalid syntax', ('<unknown>', 1,...","Rebuilding trust takes time, honest communicat...",NaN,NaN,3.327500,565721c0-d284-41b0-8b0e-83dfb2d55f6f,958e7330-643e-4cf1-bce1-e19a94827200,NaN,NaN
7,Is it normal to have doubts in a long-term rel...,NaN,"SyntaxError('invalid syntax', ('<unknown>', 1,...","Yes, occasional doubts are common and worth di...",NaN,NaN,2.871808,60d16afd-d30d-4984-a70e-8717a5c05122,8958e730-c5bf-4a46-b0fc-20d3baea23ab,NaN,NaN
8,How can I improve communication with my partner?,NaN,"SyntaxError('invalid syntax', ('<unknown>', 1,...","Practice active listening, be honest, and avoi...",NaN,NaN,3.247387,62401e1c-d8bf-4472-a106-7ec561437948,3347b726-b806-490b-a927-883b1e6a99b2,NaN,NaN
9,What are signs of a healthy relationship?,"Trust, respect, open communication, mutual sup...",None,"Trust, respect, open communication, and emotio...",True,True,2.957871,81f5c0c7-9634-40e4-8b8d-dc2fd7f8ff06,e5442a24-13b8-408a-a1f3-a4c624f408cf,NaN,NaN


In [12]:
experiment_results.to_pandas().to_csv('traces.csv')

In [ ]:
async def test_concision_score():
    experiment_results = await client.aevaluate(
        ls_target,
        data=dataset_name,
        evaluators=[concision, correctness]
    )
    feedback = client.list_feedback(
        run_ids=[r.id for r in client.list_runs(project_name=experiment_results.experiment_name)],
        feedback_key="concision"
    )
    scores = [f.score for f in feedback]
    assert sum(scores) / len(scores) >= 0.8, "Concision score below 80%"